# Your Title Here

**Name(s)**: Tom Hocquet and Julia Ma

**Website Link**: https://tomok59.github.io/power_outage_model/

## Code

In [141]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import matplotlib.pyplot as plt
pd.options.plotting.backend = 'plotly'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

### Framing the Problem

In [142]:
df = pd.read_excel('outage_copy.xlsx', header=5)
df = df.drop([0]).drop(columns='variables').set_index('OBS')
df['YEAR'] = df['YEAR'].astype(int)
df = df.reset_index()
def to_day(col):
    if str(col)[8:10] == '':
        return '00'
    return str(col)[8:10]
def to_hour(col):
    if str(col)[0:2] == 'na':
        return '25'
    return str(col)[0:2]
df['START.DAY'] = df['OUTAGE.START.DATE'].apply(to_day)
df['START.HOUR'] = df['OUTAGE.START.TIME'].apply(to_hour)
df['END.DAY'] = df['OUTAGE.RESTORATION.DATE'].apply(to_day)
df['END.HOUR'] = df['OUTAGE.RESTORATION.TIME'].apply(to_hour)
df = df.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'])
df = df.drop(columns=['HURRICANE.NAMES', 'POSTAL.CODE', 'CAUSE.CATEGORY.DETAIL', 'OBS'])
df

,YEAR,MONTH,U.S._STATE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,...,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,START.DAY,START.HOUR,END.DAY,END.HOUR
0,2011,7.0,Minnesota,MRO,East North Central,-0.3,normal,severe weather,3060,NaN,...,18.2,2.14,0.6,91.592666,8.407334,5.478743,01,17,03,20
1,2014,5.0,Minnesota,MRO,East North Central,-0.1,normal,intentional attack,1,NaN,...,18.2,2.14,0.6,91.592666,8.407334,5.478743,11,18,11,18
2,2010,10.0,Minnesota,MRO,East North Central,-1.5,cold,severe weather,3000,NaN,...,18.2,2.14,0.6,91.592666,8.407334,5.478743,26,20,28,22
3,2012,6.0,Minnesota,MRO,East North Central,-0.1,normal,severe weather,2550,NaN,...,18.2,2.14,0.6,91.592666,8.407334,5.478743,19,04,20,23
4,2015,7.0,Minnesota,MRO,East North Central,1.2,warm,severe weather,1740,250,...,18.2,2.14,0.6,91.592666,8.407334,5.478743,18,02,19,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,North Dakota,MRO,West North Central,-0.9,cold,public appeal,720,155,...,3.9,0.27,0.1,97.599649,2.401765,2.401765,06,08,06,20
1530,2006,NaN,North Dakota,MRO,West North Central,NaN,NaN,fuel supply emergency,NaN,1650,...,3.9,0.27,0.1,97.599649,2.401765,2.401765,00,25,00,25
1531,2009,8.0,South Dakota,RFC,West North Central,0.5,warm,islanding,59,84,...,4.7,0.3,0.15,98.307744,1.692256,1.692256,29,22,29,23
1532,2009,8.0,South Dakota,MRO,West North Central,0.5,warm,islanding,181,373,...,4.7,0.3,0.15,98.307744,1.692256,1.692256,29,11,29,14


In [143]:
df.isna().sum()

YEAR                    0
MONTH                   9
U.S._STATE              0
NERC.REGION             0
CLIMATE.REGION          6
ANOMALY.LEVEL           9
CLIMATE.CATEGORY        9
CAUSE.CATEGORY          0
OUTAGE.DURATION        58
DEMAND.LOSS.MW        705
CUSTOMERS.AFFECTED    443
RES.PRICE              22
COM.PRICE              22
IND.PRICE              22
TOTAL.PRICE            22
RES.SALES              22
COM.SALES              22
IND.SALES              22
TOTAL.SALES            22
RES.PERCEN             22
COM.PERCEN             22
IND.PERCEN             22
RES.CUSTOMERS           0
COM.CUSTOMERS           0
IND.CUSTOMERS           0
TOTAL.CUSTOMERS         0
RES.CUST.PCT            0
COM.CUST.PCT            0
IND.CUST.PCT            0
PC.REALGSP.STATE        0
PC.REALGSP.USA          0
PC.REALGSP.REL          0
PC.REALGSP.CHANGE       0
UTIL.REALGSP            0
TOTAL.REALGSP           0
UTIL.CONTRI             0
PI.UTIL.OFUSA           0
POPULATION              0
POPPCT_URBAN

In [144]:
na_cols = df.columns[df.isna().any()].tolist()
values_to_remove = ['CLIMATE.REGION',
 'ANOMALY.LEVEL',
 'CLIMATE.CATEGORY']
filtered_list = list(filter(lambda x: x not in values_to_remove, na_cols))
filtered_list
for colns in filtered_list:
    df[colns] = df[colns].fillna(-1)
df.isna().sum()

YEAR                  0
MONTH                 0
U.S._STATE            0
NERC.REGION           0
CLIMATE.REGION        6
ANOMALY.LEVEL         9
CLIMATE.CATEGORY      9
CAUSE.CATEGORY        0
OUTAGE.DURATION       0
DEMAND.LOSS.MW        0
CUSTOMERS.AFFECTED    0
RES.PRICE             0
COM.PRICE             0
IND.PRICE             0
TOTAL.PRICE           0
RES.SALES             0
COM.SALES             0
IND.SALES             0
TOTAL.SALES           0
RES.PERCEN            0
COM.PERCEN            0
IND.PERCEN            0
RES.CUSTOMERS         0
COM.CUSTOMERS         0
IND.CUSTOMERS         0
TOTAL.CUSTOMERS       0
RES.CUST.PCT          0
COM.CUST.PCT          0
IND.CUST.PCT          0
PC.REALGSP.STATE      0
PC.REALGSP.USA        0
PC.REALGSP.REL        0
PC.REALGSP.CHANGE     0
UTIL.REALGSP          0
TOTAL.REALGSP         0
UTIL.CONTRI           0
PI.UTIL.OFUSA         0
POPULATION            0
POPPCT_URBAN          0
POPPCT_UC             0
POPDEN_URBAN          0
POPDEN_UC       

In [145]:
df = df.dropna()
df.shape

(1520, 52)

### Baseline Model

In [146]:
## make initial split for test at the end

df_1 = df.copy()
df_X = df_1.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_1[['CAUSE.CATEGORY']]

X_train, X_test_final, y_train, y_test_final = (
    train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
)
df_1 = X_train
df_1['CAUSE.CATEGORY'] = y_train
df_1

,YEAR,MONTH,U.S._STATE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,...,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,START.DAY,START.HOUR,END.DAY,END.HOUR,CAUSE.CATEGORY
1450,2012,11.0,Massachusetts,NPCC,Northeast,0.1,normal,1,0,0.0,...,38.29,1.17,73.905628,26.094372,4.604889,15,5,15,5,intentional attack
27,2011,4.0,Tennessee,SERC,Central,-0.5,cold,660,300,63000.0,...,7.05,1.72,97.843109,2.156891,2.156891,4,13,5,0,severe weather
467,2016,5.0,Washington,WECC,Northwest,0.6,warm,-1,-1,-1.0,...,3.57,0.62,93.208786,6.791214,2.405397,8,9,0,25,islanding
1284,2014,4.0,California,WECC,West,-0.2,normal,1,-1,-1.0,...,5.28,0.59,95.164177,4.835823,1.730658,8,13,8,13,intentional attack
907,2012,1.0,Delaware,RFC,Northeast,-0.7,cold,0,0,0.0,...,20.88,6.21,78.304540,21.695460,3.656087,12,8,12,8,intentional attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,2008,9.0,Ohio,RFC,Central,-0.3,normal,11531,469,564728.0,...,10.82,2.05,91.154687,8.845313,1.057422,14,17,22,17,severe weather
910,2013,1.0,Iowa,MRO,East North Central,-0.4,normal,1637,-1,-1.0,...,1.71,0.66,99.260747,0.739253,0.739253,16,7,17,10,intentional attack
1101,2009,11.0,California,WECC,West,1.1,warm,945,630,-1.0,...,5.28,0.59,95.164177,4.835823,1.730658,18,6,18,22,equipment failure
235,2000,3.0,Texas,TRE,South,-1.1,cold,70,400,100000.0,...,3.35,0.58,97.258336,2.742036,2.090873,18,16,18,17,system operability disruption


In [147]:
# first guess of relevant columns : 'OUTAGE.DURATION', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'AREAPCT_URBAN'
model_1_df = df_1[['OUTAGE.DURATION', 'YEAR', 'MONTH', 'U.S._STATE', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'AREAPCT_URBAN', 'CAUSE.CATEGORY']]
model_1_df

,OUTAGE.DURATION,YEAR,MONTH,U.S._STATE,TOTAL.PRICE,TOTAL.SALES,TOTAL.CUSTOMERS,AREAPCT_URBAN,CAUSE.CATEGORY
1450,1,2012,11.0,Massachusetts,13.05,4294884,3109560.0,38.29,intentional attack
27,660,2011,4.0,Tennessee,8.91,7278927,3174123.0,7.05,severe weather
467,-1,2016,5.0,Washington,7.48,6669170,3382664.0,3.57,islanding
1284,1,2014,4.0,California,11.74,19847009,15079226.0,5.28,intentional attack
907,0,2012,1.0,Delaware,10.78,990197,451908.0,20.88,intentional attack
...,...,...,...,...,...,...,...,...,...
718,11531,2008,9.0,Ohio,8.54,12821117,5525586.0,10.82,severe weather
910,1637,2013,1.0,Iowa,7.73,4132984,1572284.0,1.71,intentional attack
1101,945,2009,11.0,California,11.36,19159063,14789510.0,5.28,equipment failure
235,70,2000,3.0,Texas,5.91,21870637,9299829.0,3.35,system operability disruption


In [148]:
X_train, X_test, y_train, y_test = (
    train_test_split(model_1_df[['OUTAGE.DURATION', 'YEAR', 'MONTH', 'U.S._STATE', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'AREAPCT_URBAN']], model_1_df['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
)

In [149]:
scaler = StandardScaler()
neigh = KNeighborsClassifier(n_neighbors=10)
preprocessor = ColumnTransformer(
        transformers=[
            ('STD_Sales', scaler, ['TOTAL.SALES']),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH']),
        ],
        remainder='passthrough'
    )
pipeline = Pipeline([
        ('prepocessor', preprocessor),
        ('knn', neigh)
    ])
pipeline.fit(X_train, y_train)

print('Train Error: ' + str(1-pipeline.score(X_train, y_train)))
print('Test Error: ' + str(1-pipeline.score(X_test, y_test)))

Train Error: 0.33684210526315794
Test Error: 0.39649122807017545


In [150]:
preds_knn = pipeline.predict(X_train)
f1_score(preds_knn, y_train, average = 'weighted')

0.7108217463207122

In [151]:
preds_knn = pipeline.predict(X_test)
f1_score(preds_knn, y_test, average = 'weighted')

0.6438866343750854

### Final Model

In [152]:
hyperparameters_knn = {
    'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10,15,20,25],
    'knn__weights' : ['uniform', 'distance'],
    'knn__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knn__p': [1,2]
}
searcher = GridSearchCV(pipeline, hyperparameters_knn, cv=5)
searcher.fit(X_train, y_train)
searcher.best_params_

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_base.py:504: UserWarning:

cannot use tree with sparse input: using brute force

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_base.py:504: UserWarning:

cannot use tree with sparse input: using brute force

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_base.py:504: UserWarning:

cannot use tree with sparse input: using brute force

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_base.py:504: UserWarning:

cannot use tree with sparse input: using brute force

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_base.py:504: UserWarning:

cannot use tree with sparse input: using brute force

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_base.py:504: UserWarning:

cannot use tree with sparse input: using brute force

/Users/demo/anaconda3/envs/dsc80/l

{'knn__algorithm': 'auto',
 'knn__n_neighbors': 25,
 'knn__p': 1,
 'knn__weights': 'distance'}

In [153]:
best_params = {key[5:]: value for key, value in searcher.best_params_.items()}
neigh_v2 = KNeighborsClassifier(**best_params)
preprocessor_knn_v2 = ColumnTransformer(
        transformers=[
            ('STD_Sales', scaler, ['TOTAL.SALES']),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH']),
        ],
        remainder='passthrough'
    )
pipeline_knn_v2 = Pipeline([
        ('prepocessor', preprocessor_knn_v2),
        ('knn', neigh_v2)
    ])
pipeline_knn_v2.fit(X_train, y_train)
print('Train Error: ' + str(1-pipeline_knn_v2.score(X_train, y_train)))
print('Test Error: ' + str(1-pipeline_knn_v2.score(X_test, y_test)))


Train Error: 0.002339181286549752
Test Error: 0.4


Error went down 3% by finding best parameters

tree for each column with depth 5, sorted by f1score min error

In [155]:
df_2 = df_1.copy()
df_X = df_2.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_2['CAUSE.CATEGORY']

column_name = []
tr_err = []
ts_err = []
f1_Scores_TR = []
f1_Scores_TS = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
    )

for col in numeric_columns:
    std_scaler = StandardScaler()
    tree = RandomForestClassifier(max_depth=5)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', [col])
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    pipeline_tree.fit(X_train, y_train)
    column_name.append(col + ' NUM')
    tr_err.append(1-pipeline_tree.score(X_train, y_train))
    ts_err.append(1-pipeline_tree.score(X_test, y_test))
    preds_TR = pipeline_tree.predict(X_train)
    f1_Scores_TR.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
    preds_TS = pipeline_tree.predict(X_test)
    f1_Scores_TS.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

for col in categorical_columns:
    std_scaler = StandardScaler()
    tree = RandomForestClassifier(max_depth=5)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    pipeline_tree.fit(X_train, y_train)
    column_name.append(col + ' CAT')
    tr_err.append(1-pipeline_tree.score(X_train, y_train))
    ts_err.append(1-pipeline_tree.score(X_test, y_test))
    preds_TR = pipeline_tree.predict(X_train)
    f1_Scores_TR.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
    preds_TS = pipeline_tree.predict(X_test)
    f1_Scores_TS.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors['F1_train_err'] = f1_Scores_TR
errors['F1_test_err'] = f1_Scores_TS
errors.sort_values(by='F1_test_err')

,column,train_err,test_err,F1_train_err,F1_test_err
5,CUSTOMERS.AFFECTED NUM,0.242105,0.256140,0.192130,0.208928
32,POPULATION NUM,0.346199,0.357895,0.272665,0.286625
37,POPDEN_RURAL NUM,0.397661,0.378947,0.309219,0.294345
18,COM.CUSTOMERS NUM,0.370760,0.371930,0.297248,0.294496
29,TOTAL.REALGSP NUM,0.388304,0.375439,0.300511,0.296993
34,POPPCT_UC NUM,0.397661,0.378947,0.311057,0.300515
36,POPDEN_UC NUM,0.409357,0.389474,0.320777,0.303476
35,POPDEN_URBAN NUM,0.401170,0.389474,0.310434,0.304196
20,TOTAL.CUSTOMERS NUM,0.364912,0.375439,0.285327,0.306064
39,AREAPCT_UC NUM,0.402339,0.382456,0.319463,0.307241


We can see our best column is 'CUSTOMERS.AFFECTED'. Let's combine that column with other ones to see the scores we get!

In [156]:
df_2 = df_1.copy()
df_X = df_2.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_2['CAUSE.CATEGORY']

column_name = []
tr_err = []
ts_err = []
f1_Scores_TR = []
f1_Scores_TS = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
    )
col1 = ['CUSTOMERS.AFFECTED']
for col in numeric_columns:
    col1 = ['CUSTOMERS.AFFECTED']
    col1.append(col)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1)
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    column_name.append(str(col1) + ' NUM')
    temp_train = []
    temp_test =[]
    temp_f1_tr =[]
    temp_f1_ts = []
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

        preds_TR = pipeline_tree.predict(X_train)
        temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
        preds_TS = pipeline_tree.predict(X_test)
        temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    f1_Scores_TR.append(np.mean(temp_f1_tr))
    f1_Scores_TS.append(np.mean(temp_f1_ts))

for col in categorical_columns:
    col1 = ['CUSTOMERS.AFFECTED']
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1),
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ]) 
    column_name.append(str(col1) + col + ' CAT')
    temp_train = []
    temp_test =[]
    temp_f1_tr =[]
    temp_f1_ts = []
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

        preds_TR = pipeline_tree.predict(X_train)
        temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
        preds_TS = pipeline_tree.predict(X_test)
        temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    f1_Scores_TR.append(np.mean(temp_f1_tr))
    f1_Scores_TS.append(np.mean(temp_f1_ts))

errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors['F1_train_err'] = f1_Scores_TR
errors['F1_test_err'] = f1_Scores_TS
errors.sort_values(by='F1_test_err')

,column,train_err,test_err,F1_train_err,F1_test_err
25,"['CUSTOMERS.AFFECTED', 'PC.REALGSP.USA'] NUM",0.131345,0.225263,0.117833,0.207522
47,['CUSTOMERS.AFFECTED']U.S._STATE CAT,0.190994,0.252982,0.163547,0.212370
0,"['CUSTOMERS.AFFECTED', 'YEAR'] NUM",0.130994,0.227719,0.118967,0.213224
13,"['CUSTOMERS.AFFECTED', 'TOTAL.SALES'] NUM",0.005497,0.226316,0.005474,0.215502
32,"['CUSTOMERS.AFFECTED', 'POPULATION'] NUM",0.052982,0.224561,0.053207,0.218238
7,"['CUSTOMERS.AFFECTED', 'COM.PRICE'] NUM",0.033333,0.240702,0.033074,0.224972
4,"['CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW'] NUM",0.113567,0.261754,0.103330,0.227912
10,"['CUSTOMERS.AFFECTED', 'RES.SALES'] NUM",0.004211,0.237193,0.004183,0.228197
6,"['CUSTOMERS.AFFECTED', 'RES.PRICE'] NUM",0.031111,0.238596,0.030784,0.229800
22,"['CUSTOMERS.AFFECTED', 'COM.CUST.PCT'] NUM",0.052749,0.235789,0.052781,0.232451


In [157]:
df_2 = df_1.copy()
df_X = df_2.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_2['CAUSE.CATEGORY']

column_name = []
tr_err = []
ts_err = []
f1_Scores_TR = []
f1_Scores_TS = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
    )
col1 = ['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP']
for col in numeric_columns:
    col1 = ['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP']
    col1.append(col)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1)
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    column_name.append(str(col1) + ' NUM')
    temp_train = []
    temp_test =[]
    temp_f1_tr =[]
    temp_f1_ts = []
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

        preds_TR = pipeline_tree.predict(X_train)
        temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
        preds_TS = pipeline_tree.predict(X_test)
        temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    f1_Scores_TR.append(np.mean(temp_f1_tr))
    f1_Scores_TS.append(np.mean(temp_f1_ts))

for col in categorical_columns:
    col1 = ['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP']
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1),
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ]) 
    column_name.append(str(col1) + col + ' CAT')
    temp_train = []
    temp_test =[]
    temp_f1_tr =[]
    temp_f1_ts = []
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

        preds_TR = pipeline_tree.predict(X_train)
        temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
        preds_TS = pipeline_tree.predict(X_test)
        temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    f1_Scores_TR.append(np.mean(temp_f1_tr))
    f1_Scores_TS.append(np.mean(temp_f1_ts))

errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors['F1_train_err'] = f1_Scores_TR
errors['F1_test_err'] = f1_Scores_TS
errors.sort_values(by='F1_test_err')

,column,train_err,test_err,F1_train_err,F1_test_err
7,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'COM.P...",0.003509,0.189825,0.003491,0.170068
9,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'TOTAL...",0.003626,0.191930,0.003619,0.178245
2,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'ANOMA...",0.008772,0.190175,0.008710,0.179616
14,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'RES.P...",0.003860,0.194035,0.003846,0.182060
6,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'RES.P...",0.003509,0.196491,0.003506,0.187520
3,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'OUTAG...",0.001170,0.192632,0.001165,0.190654
11,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'COM.S...",0.003626,0.205614,0.003622,0.191489
8,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'IND.P...",0.003743,0.206316,0.003722,0.195040
1,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'MONTH...",0.003509,0.211579,0.003502,0.201524
47,"['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP']U.S._ST...",0.106316,0.218596,0.101627,0.204657


In [159]:
errors = errors.sort_values(by='F1_test_err')
errors.reset_index()
errors.iloc[0:10]['column'].to_list()

["['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'COM.PRICE'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'TOTAL.PRICE'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'ANOMALY.LEVEL'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'RES.PERCEN'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'RES.PRICE'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'OUTAGE.DURATION'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'COM.SALES'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'IND.PRICE'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'MONTH'] NUM",
 "['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP']U.S._STATE CAT"]

In [160]:
df_X[['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'TOTAL.PRICE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'RES.PERCEN', 'U.S._STATE', 'MONTH']]

,CUSTOMERS.AFFECTED,TOTAL.REALGSP,TOTAL.PRICE,ANOMALY.LEVEL,OUTAGE.DURATION,RES.PERCEN,U.S._STATE,MONTH
1450,0.0,414029,13.05,0.1,1,35.491552,Massachusetts,11.0
27,63000.0,258910,8.91,-0.5,660,38.324811,Tennessee,4.0
467,-1.0,420809,7.48,0.6,-1,34.589237,Washington,5.0
1284,-1.0,2118858,11.74,-0.2,1,30.968057,California,4.0
907,0.0,57146,10.78,-0.7,0,43.151817,Delaware,1.0
...,...,...,...,...,...,...,...,...
718,564728.0,502597,8.54,-0.3,11531,30.376495,Ohio,9.0
910,-1.0,149313,7.73,-0.4,1637,37.138445,Iowa,1.0
1101,-1.0,1913674,11.36,1.1,945,31.955034,California,11.0
235,100000.0,944631,5.91,-1.1,70,29.754232,Texas,3.0


In [161]:
df_2 = df_1.copy()
df_X = df_2.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_2['CAUSE.CATEGORY']


X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
    )




col1 = ['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'TOTAL.PRICE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'RES.PERCEN', 'MONTH']
tree = RandomForestClassifier(max_depth=15)
preprocessor_tree = ColumnTransformer(
        transformers=[
            ('column', 'passthrough', col1),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'CLIMATE.CATEGORY'])
        ],
        remainder='drop'
    )
pipeline_tree = Pipeline([
        ('prepocessor', preprocessor_tree),
        ('rf', tree)
    ])





pipeline_tree.fit(X_train, y_train)

preds_TR = pipeline_tree.predict(X_train)
preds_TS = pipeline_tree.predict(X_test)

print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
print('F1 Train Error: ' + str(1-f1_score(preds_TR, y_train, average = 'weighted')))
print('F1 Test Error: ' + str(1-f1_score(preds_TS, y_test, average = 'weighted')))

Train Error: 0.007017543859649145
Test Error: 0.1719298245614035
F1 Train Error: 0.0070024243632164085
F1 Test Error: 0.15320934591228963


In [168]:
df_2 = df_1.copy()
df_X = df_2.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_2['CAUSE.CATEGORY']


X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
    )

temp_f1_tr = []
temp_f1_ts = []
temp_train = []
temp_test = []
depth = []
for i in range(25):
    col1 = ['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'TOTAL.PRICE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'RES.PERCEN', 'MONTH']
    tree = RandomForestClassifier(max_depth=i+1)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1),
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'CLIMATE.CATEGORY'])
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])

    pipeline_tree.fit(X_train, y_train)
    depth.append(i+1)
    temp_train.append(1-pipeline_tree.score(X_train, y_train))
    temp_test.append(1-pipeline_tree.score(X_test, y_test))

    preds_TR = pipeline_tree.predict(X_train)
    temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
    preds_TS = pipeline_tree.predict(X_test)
    temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

In [169]:
pd.DataFrame({'train': temp_f1_tr, 'test': temp_f1_ts}, index=depth).plot().update_layout(
    xaxis_title='max_depth', yaxis_title='F1 error'
)

In [110]:
hyperparameters_tree ={
    #'rf__n_estimators': [10, 50, 100, 200, 300, 400, 500, 1000],
    'rf__criterion': ["gini", "entropy"],
    'rf__max_depth': [10, 15, 20, 25, 50],
    'rf__min_samples_split': [1.0, 2, 4, 8, 10],
    #'rf__min_samples_leaf': [1, 2, 4, 6, 8, 10],
    'rf__max_features': ["auto", "log2", None]
}
searcher_tree = GridSearchCV(pipeline_tree, hyperparameters_tree, cv=5)
searcher_tree.fit(X_train, y_train)
searcher_tree.best_params_

{'rf__criterion': 'entropy',
 'rf__max_depth': 25,
 'rf__max_features': None,
 'rf__min_samples_split': 2}

In [172]:
df_2 = df_1.copy()
df_X = df_2.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_2['CAUSE.CATEGORY']


X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_Y,test_size=0.25, random_state=1)
    )


col1 = ['CUSTOMERS.AFFECTED', 'TOTAL.REALGSP', 'TOTAL.PRICE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'RES.PERCEN', 'MONTH']
tree = RandomForestClassifier(criterion='entropy', max_depth=15)
preprocessor_tree = ColumnTransformer(
        transformers=[
            ('column', 'passthrough', col1),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'CLIMATE.CATEGORY'])
        ],
        remainder='drop'
    )
pipeline_tree = Pipeline([
        ('prepocessor', preprocessor_tree),
        ('rf', tree)
    ])

pipeline_tree.fit(X_train, y_train)

preds_TR = pipeline_tree.predict(X_train)
preds_TS = pipeline_tree.predict(X_test)

print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
print('F1 Train Error: ' + str(1-f1_score(preds_TR, y_train, average = 'weighted')))
print('F1 Test Error: ' + str(1-f1_score(preds_TS, y_test, average = 'weighted')))

Train Error: 0.0008771929824561431
Test Error: 0.18684210526315792
F1 Train Error: 0.0008723660810149303
F1 Test Error: 0.15690592477710563


In [173]:
final_preds = pipeline_tree.predict(X_test_final)
print('F1 Test Error: ' + str(1-f1_score(final_preds, y_test_final, average = 'weighted')))

F1 Test Error: 0.15690592477710563


### Fairness Analysis

In [ ]:
# TODO